In [1]:
with open('day1problem1.txt', 'r') as f:
    inputs = f.read().splitlines()

### part 1

In [2]:
len(inputs)

4136

In [3]:
inputs[:4]

['L10', 'L24', 'R17', 'R48']

In [4]:
def crack_password(inputs):
    zeros_cnter = 0
    curr_pos = 50
    for i in inputs:
        dir = 1 if i[0] == 'R' else -1
        mag = int(i[1:])
        curr_pos = curr_pos + dir*(mag%100)
        if curr_pos>=0:
            curr_pos = curr_pos%100
        else:
            curr_pos = 100 + curr_pos
        if curr_pos == 0:
            zeros_cnter+=1
    return zeros_cnter

crack_password(inputs)

1007

### part 2 

In [5]:
inputs[:5]

['L10', 'L24', 'R17', 'R48', 'L36']

In [6]:
def crack_password_2(inputs):
    zeros_cnter = 0
    curr_pos = 50
    is_last_zero = False
    for i in inputs:
        dir = 1 if i[0] == 'R' else -1
        mag = int(i[1:])
        zeros_cnter += int(mag//100)
        curr_pos = curr_pos + dir*(mag%100)
        if curr_pos>0:
            zeros_cnter += int(curr_pos//100)
            curr_pos = curr_pos%100
        elif curr_pos<0:
            if not is_last_zero:
                zeros_cnter += 1
            curr_pos = 100 + curr_pos
        else:
            zeros_cnter+=1
        if curr_pos == 0:
            is_last_zero = True
        else:
            is_last_zero = False
        # print(curr_pos, i, zeros_cnter)
    return zeros_cnter

crack_password_2(inputs)

5820

### testing on huge inputs

In [7]:
with open('day01_large.txt', 'r') as f:
    big_inputs = f.read().splitlines()

In [8]:
len(big_inputs)

10000000

In [10]:
%%time
crack_password(big_inputs)

CPU times: user 1.65 s, sys: 10.9 ms, total: 1.67 s
Wall time: 1.67 s


100329

In [11]:
%%time
crack_password_2(big_inputs)

CPU times: user 2.57 s, sys: 15.3 ms, total: 2.58 s
Wall time: 2.58 s


50047983

### Use sonnet 4.5 opus to write faster code

In [50]:
def crack_password_cursor(inputs):
    zeros_cnter = 0
    curr_pos = 50
    MOD = 100

    # Precompute directions and magnitudes in a faster vectorized loop
    dirs = [1 if instr[0] == 'R' else -1 for instr in inputs]
    mags = [int(instr[1:]) for instr in inputs]
    for d, m in zip(dirs, mags):
        move = d * (m % MOD)
        curr_pos = (curr_pos + move) % MOD
        if curr_pos == 0:
            zeros_cnter += 1
    return zeros_cnter

crack_password_cursor(inputs)

1007

In [53]:
def crack_password_2_cursor(inputs):
    zeros_cnter = 0
    curr_pos = 50
    MOD = 100

    for instr in inputs:
        dir = 1 if instr[0] == 'R' else -1
        mag = int(instr[1:])

        move = dir * mag
        # Instead of iterating every step, we can compute how many positions modulo 100 cross zero:
        # The start is curr_pos, end is curr_pos + move
        # For step = 1 to abs(move): 
        #   (curr_pos + dir * step) % MOD == 0
        # Solve for step in 1..abs(move), where curr_pos + dir*step ≡ 0 mod MOD
        # So there is a zero crossing if there exists a step in 1..abs(move) such that
        #   dir*step ≡ -curr_pos mod MOD
        #   step ≡ (-curr_pos * dir) mod MOD

        # step (positive, <= abs(move))
        cand_step = (-curr_pos * dir) % MOD
        if 1 <= cand_step <= abs(move):
            zeros_cnter += 1
        elif abs(move) >= MOD:
            # If move skips at least a full cycle, guarantee at least one zero crossing per full cycle
            # Number of crossings = number of full MOD traversals in |move|
            # However, because each move starts from curr_pos and only first crossing is counted in each move,
            # for this puzzle it always at most one crossing per move, because only one path from curr_pos to new pos
            # Let’s count all zeros between curr_pos (exclusive) and curr_pos+move (inclusive), wrapping around MOD.
            # This is equivalent to checking if 0 is in the open interval (curr_pos, curr_pos + move] mod MOD

            # For backwards moves, swap the limits
            start = curr_pos
            end = (curr_pos + move) % MOD
            # Use circular interval inclusion
            # For move > 0: 0 in (start, end] mod MOD  ⇔ start < 0 <= end or end < start < 0 or 0 <= end < start
            # For move < 0: 0 in (end, start] mod MOD

            # Instead, actually just always check if range wraps or covers zero.
            # Number of times position 0 is passed: floor((curr_pos + move) / MOD) - floor(curr_pos / MOD)
            # But curr_pos can be in [0, MOD), move can be positive or negative
            num_crosses = 0
            if move > 0:
                cross_before = (MOD - curr_pos - 1) // MOD
                cross_after = (MOD - (curr_pos + move) - 1) // MOD
                num_crosses = ((curr_pos + move) // MOD) - (curr_pos // MOD)
            elif move < 0:
                # Backwards. curr_pos decreases
                cross_before = curr_pos // MOD
                cross_after = (curr_pos + move) // MOD
                num_crosses = (curr_pos // MOD) - ((curr_pos + move) // MOD)
            else:
                num_crosses = 0
            # But with all positions mod MOD, either way, for spans >= MOD, always cross zero once
            if num_crosses >= 1:
                zeros_cnter += 1
        # End move at new position
        curr_pos = (curr_pos + move) % MOD

    return zeros_cnter

crack_password_2_cursor(inputs)

2402

In [51]:
%%timeit
crack_password_cursor(big_inputs)

1.75 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%%time
crack_password_2_cursor(big_inputs)

CPU times: user 2.19 s, sys: 11.8 ms, total: 2.2 s
Wall time: 2.2 s


9504752